In [ ]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras import layers, Model, Input


In [ ]:
DATA_DIR = "img_align_celeba"   
OUTPUT_DIR = "generate_face"    
os.makedirs(OUTPUT_DIR, exist_ok=True)

BATCH_SIZE = 128
IMG_SIZE = 64
Z_DIM = 100
EPOCHS = 100

In [ ]:
print("Loading dataset...")

dataset = tf.keras.preprocessing.image_dataset_from_directory(
    directory=DATA_DIR,
    label_mode=None,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=42,
)

In [ ]:
dataset = dataset.map(lambda x: (x / 127.5) - 1.0)
dataset = dataset.prefetch(tf.data.AUTOTUNE)


In [ ]:
def build_discriminator(img_shape=(IMG_SIZE, IMG_SIZE, 3)):
    inp = Input(shape=img_shape)

    x = layers.Conv2D(64, 4, strides=2, padding="same")(inp)
    x = layers.LeakyReLU(0.2)(x)
    x = layers.Dropout(0.3)(x)

    x = layers.Conv2D(128, 4, strides=2, padding="same")(x)
    x = layers.LeakyReLU(0.2)(x)
    x = layers.Dropout(0.3)(x)

    x = layers.Conv2D(256, 4, strides=2, padding="same")(x)
    x = layers.LeakyReLU(0.2)(x)
    x = layers.Dropout(0.3)(x)

    x = layers.Conv2D(512, 4, strides=2, padding="same")(x)
    x = layers.LeakyReLU(0.2)(x)
    x = layers.Dropout(0.3)(x)

    x = layers.Flatten()(x)
    out = layers.Dense(1, activation="sigmoid")(x)

    model = Model(inp, out)
    return model

discriminator = build_discriminator()
discriminator.summary()


In [ ]:
def build_generator(z_dim=Z_DIM):
    inp = Input(shape=(z_dim,))

    x = layers.Dense(8 * 8 * 256, use_bias=False)(inp)
    x = layers.Reshape((8, 8, 256))(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(0.2)(x)

    x = layers.Conv2DTranspose(128, 4, strides=2, padding="same", use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(0.2)(x)

    x = layers.Conv2DTranspose(64, 4, strides=2, padding="same", use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(0.2)(x)

    x = layers.Conv2DTranspose(3, 4, strides=2, padding="same", activation="tanh")(x)

    model = Model(inp, x)
    return model

generator = build_generator()
generator.summary()


In [ ]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=False)

disc_optimizer = tf.keras.optimizers.Adam(learning_rate=2e-4, beta_1=0.5)
gen_optimizer = tf.keras.optimizers.Adam(learning_rate=2e-4, beta_1=0.5)


In [ ]:
@tf.function
def train_step(real_images):
    batch_size = tf.shape(real_images)[0]
    noise = tf.random.normal([batch_size, Z_DIM])

    with tf.GradientTape() as disc_tape, tf.GradientTape() as gen_tape:
        fake_images = generator(noise, training=True)

        real_output = discriminator(real_images, training=True)
        fake_output = discriminator(fake_images, training=True)

        real_labels = tf.ones_like(real_output) * 0.9
        fake_labels = tf.zeros_like(fake_output)

        d_loss_real = cross_entropy(real_labels, real_output)
        d_loss_fake = cross_entropy(fake_labels, fake_output)
        disc_loss = d_loss_real + d_loss_fake

        gen_labels = tf.ones_like(fake_output)
        gen_loss = cross_entropy(gen_labels, fake_output)

    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)

    disc_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    gen_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))

    return disc_loss, gen_loss


In [ ]:
def denormalize(img):
    return (img + 1.0) / 2.0

def generate_images(epoch, seed):
    predictions = generator(seed, training=False)
    predictions = denormalize(predictions)

    fig = plt.figure(figsize=(4, 4))
    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i + 1)
        plt.imshow(predictions[i])
        plt.axis("off")

    path = os.path.join(OUTPUT_DIR, f"samples_epoch_{epoch:03d}.png")
    plt.savefig(path)
    plt.close(fig)
    print(f"Saved: {path}")


seed = tf.random.normal([16, Z_DIM])


In [ ]:
def train(dataset, epochs):
    for epoch in range(epochs):
        print(f"\n===== Epoch {epoch+1}/{epochs} =====")
        for step, real_images in enumerate(dataset):
            d_loss, g_loss = train_step(real_images)

            if step % 100 == 0:
                print(f"Step {step} | D loss: {d_loss.numpy():.4f} | G loss: {g_loss.numpy():.4f}")

        generate_images(epoch, seed)


In [ ]:
if __name__ == "__main__":
    print("Training DCGAN...")
    train(dataset, EPOCHS)
    print("Training complete.")
